In [1]:
import torch
import torch.nn as nn
from torch.nn import functional as F

In [2]:
%set_env CUDA_VISIBLE_DEVICES=1
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(device)

env: CUDA_VISIBLE_DEVICES=1
cuda


In [20]:
# Hyperparameters
block_size = 8
batch_size = 4
max_iters = 100000 # 1000
# eval_interval = 2500
learning_rate = 3e-4
eval_iters = 250

In [4]:
with open('data/wizard_of_oz.txt', 'r', encoding='utf-8') as f:
    text = f.read()
chars = sorted(set(text))
print(chars)
text_size = len(text)
vocab_size = len(chars)

['\n', ' ', '!', '"', '&', "'", '(', ')', '*', ',', '-', '.', '0', '1', '2', '3', '4', '5', '6', '7', '8', '9', ':', ';', '?', 'A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M', 'N', 'O', 'P', 'Q', 'R', 'S', 'T', 'U', 'V', 'W', 'X', 'Y', 'Z', '[', ']', '_', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z']


In [5]:
# Encoder / Tokenizer & Decoder
string_to_int = { ch:i for i,ch in enumerate(chars) }
int_to_string = { i:ch for i,ch in enumerate(chars) }
encode = lambda s: [string_to_int[c] for c in s]
decode = lambda l: ''.join([int_to_string[i] for i in l])

data = torch.tensor(encode(text), dtype=torch.long)
# print(data[:100])

In [6]:
# split tra/val
n = int(0.8*len(data))
train_data = data[:n]
val_data = data[n:]

# generate batch with shape (batch_size, block_size)
# as well as target with the same shape
def get_batch(split):
    data = train_data if split == 'train' else val_data
    ix = torch.randint(len(data) - block_size, (batch_size,))
    x = torch.stack([data[i:i+block_size] for i in ix])
    y = torch.stack([data[i+1:i+block_size+1] for i in ix])
    x, y = x.to(device), y.to(device)
    return x, y

x, y = get_batch('train')
print('inputs:')
# print(x.shape)
print(x)
print('targets:')
print(y)

inputs:
tensor([[ 1,  1,  1,  1,  1,  1,  1,  1],
        [ 1, 54, 72, 73, 68, 67, 62, 72],
        [54, 57, 57, 58, 67, 58, 57,  1],
        [73, 68,  1, 68, 74, 71,  1, 65]], device='cuda:0')
targets:
tensor([[ 1,  1,  1,  1,  1,  1,  1,  1],
        [54, 72, 73, 68, 67, 62, 72, 61],
        [57, 57, 58, 67, 58, 57,  1, 55],
        [68,  1, 68, 74, 71,  1, 65, 54]], device='cuda:0')


In [10]:
class BigramLanguageModel(nn.Module):
    def __init__(self, vocab_size):
        super().__init__()
        self.token_embedding_table = nn.Embedding(vocab_size, vocab_size)
        
    def forward(self, index, targets=None):
        logits = self.token_embedding_table(index)
    
        if targets is None:
            loss = None
        else:
            B, T, C = logits.shape
            logits = logits.view(B*T, C)
            targets = targets.view(B*T)
            loss = F.cross_entropy(logits, targets)
        
        return logits, loss
    
    def generate(self, index, max_new_tokens):
        # index is (B, T) array of indices in the current context
        for _ in range(max_new_tokens):
            # get the predictions
            logits, loss = self.forward(index)
            # focus only on the last time step
            logits = logits[:, -1, :] # becomes (B, C)
            # apply softmax to get probabilities
            probs = F.softmax(logits, dim=-1) # (B, C)
            # sample from the distribution
            index_next = torch.multinomial(probs, num_samples=1) # (B, 1)
            # append sampled index to the running sequence
            index = torch.cat((index, index_next), dim=1) # (B, T+1)
        return index

model = BigramLanguageModel(vocab_size).to(device)
# m = model.to(device)

# context = torch.zeros((1,1), dtype=torch.long, device=device)
# generated_chars = decode(m.generate(context, max_new_tokens=500)[0].tolist())
# print(generated_chars)

In [11]:
@torch.no_grad()
def estimate_loss():
    out = {}
    model.eval()
    for split in ['train', 'val']:
        losses = torch.zeros(eval_iters)
        for k in range(eval_iters):
            X, Y = get_batch(split)
            logits, loss = model(X, Y)
            losses[k] = loss.item()
        out[split] = losses.mean()
    model.train()
    return out

In [21]:
# create a PyTorch optimizer
optimizer = torch.optim.AdamW(model.parameters(), lr=learning_rate)

for iter in range(max_iters):
    if iter % eval_iters == 0:
        losses = estimate_loss()
        print(f"step: {iter}, train loss: {losses['train']:.3f}, val loss: {losses['val']:.3f}")

    # sample a batch of data
    xb, yb = get_batch('train')

    # evaluate the loss
    logits, loss = model.forward(xb, yb)
    optimizer.zero_grad(set_to_none=True)
    loss.backward()
    optimizer.step()
print(loss.item())

step: 0, train loss: 2.536, val loss: 2.573
step: 250, train loss: 2.573, val loss: 2.587
step: 500, train loss: 2.525, val loss: 2.589
step: 750, train loss: 2.541, val loss: 2.570
step: 1000, train loss: 2.532, val loss: 2.559
step: 1250, train loss: 2.555, val loss: 2.591
step: 1500, train loss: 2.551, val loss: 2.553
step: 1750, train loss: 2.548, val loss: 2.556
step: 2000, train loss: 2.557, val loss: 2.551
step: 2250, train loss: 2.537, val loss: 2.554
step: 2500, train loss: 2.535, val loss: 2.565
step: 2750, train loss: 2.556, val loss: 2.544
step: 3000, train loss: 2.512, val loss: 2.551
step: 3250, train loss: 2.506, val loss: 2.564
step: 3500, train loss: 2.530, val loss: 2.566
step: 3750, train loss: 2.510, val loss: 2.567
step: 4000, train loss: 2.506, val loss: 2.556
step: 4250, train loss: 2.520, val loss: 2.558
step: 4500, train loss: 2.501, val loss: 2.544
step: 4750, train loss: 2.499, val loss: 2.555
step: 5000, train loss: 2.523, val loss: 2.561
step: 5250, train l

In [22]:
context = torch.zeros((1,1), dtype=torch.long, device=device)
generated_chars = decode(model.generate(context, max_new_tokens=500)[0].tolist())
print(generated_chars)


casocas sof h buid.
heruieghthe ar wad st whucoo rmeist I foearotsex t all thaner dfudnoram thinon ind arede bly ms, hes?"
neflun in, f thepute. th, ieinofu ite am a abof outhars t mun
of ale
"
suso s
" ly tha crgofagisen voved, I ma theine t wonstha s ast DEE, anind hesad wan lig  va rendfeyofocerlan thea pl, suridizy ty allo, 'le k asesth.
t d the bll.


m, y'mathed toved laitoun isthal. bon."an t stofa Julle, s ngoum gsoth, wad nd Prd ceyofoffr of tl. tould sewhet splofeimind se antou aswal t
